In [2]:
import pandas as pd
from pymongo import MongoClient
import geopandas as gpd
import sys
sys.path.append("../") # go to parent dir
from src.data_cleaning_functions import asGeoJSON, finding

In [ ]:
client = MongoClient("mongodb://localhost/geo")
db = client.get_database()

In [ ]:
companies = list(db["crunch"].find({"founded_year": {"$gt": 2003}},{"name":1,"offices":1,"category_code":1}))

In [ ]:
df = pd.DataFrame(companies)

In [ ]:
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)
cleanData = cleanData.drop(columns=["_id","offices"])

In [ ]:
cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)

In [ ]:
cleanData = cleanData.drop(columns=["latitude","longitude"])

In [ ]:
cleanData.to_json("./cleaned_offices.json", orient="records")

In [ ]:
starbucks = pd.read_csv("../input/starbucksLocation.csv")

In [ ]:
starbucks["location"] = starbucks[["Latitude","Longitude"]].apply(lambda x:asGeoJSON(x.Latitude,x.Longitude), axis=1)
cleanStarbucks = starbucks.drop(columns=["Latitude","Longitude"])
cleanStarbucks.to_json("./cleaned_starbucks.json", orient="records")

In [ ]:
airports = pd.read_csv("../input/GlobalAirportDatabase.txt", sep=":", names=["ICAO Code", "IATA Code", "Airport Name", "City/Town", "Country", "Latitude Degrees", "Latitude Minutes", "Latitude Seconds", "Latitude Direction", "Longitude Degrees", "Longitude Minutes", "Longitude Seconds", "Longitude Direction", "Altitude", "Latitude", "Longitude"])

In [ ]:
airports.loc[airports["Airport Name"] == "BARAJAS"]

In [ ]:
airports = airports.drop(columns=["ICAO Code", "IATA Code", "Latitude Degrees", "Latitude Minutes", "Latitude Seconds", "Latitude Direction", "Longitude Degrees", "Longitude Minutes", "Longitude Seconds", "Longitude Direction", "Altitude"])

In [ ]:
airports = airports.drop(airports[airports.Latitude == 0.0].index)

In [ ]:
airports["location"] = airports[["Latitude","Longitude"]].apply(lambda x:asGeoJSON(x.Latitude,x.Longitude), axis=1)
cleanAirports = airports.drop(columns=["Latitude","Longitude"])
cleanAirports.to_json("./cleaned_airports.json", orient="records")

In [ ]:
df = finding(starbucksDistance = 200) # You can change the maximum distance to a Starbucks or an Airport by passing the following args:
               # starbucksDistance = {distance in meters}, airportDistance = {distance in meters}

In [ ]:
df = df.sort_values(by="Airports_Close", ascending=False)
gdf = gpd.GeoDataFrame(df)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
gdf = gdf.drop(columns=["Latitude", "Longitude"])

In [ ]:
gdf.to_file("./offices.geojson", driver='GeoJSON')